In [11]:
import numpy as np
import pandas as pd
import os
import glob

In [12]:
import sys
sys.path.append("../pyfiles/")
from util import pickle_load

# Introduction
---
In this notebook, we like to examine the models' outputs with quantitative results. Prior to proceeding with the notebook, make sure you created the result with at least one of the notebooks below.

- `01-test_Conventional_SingleGAN.ipynb`
- `02-test_SingleGAN_soloD.ipynb`
- `03-test_Style-Restricted_GAN_nopretraining.ipynb`
- `05-test_Style-Restricted_GAN.ipynb`

---

# Preparation
--- 
In the preparation phase, we need to define some variables and call the results.

- `fe_list`: the name list for feature extractors
- `metrics`: the name list for evaluation metrics

---

In [13]:
# some variables
fe_list = ["vgg-initialization", "vgg-ImageNet", "vgg-CelebA"]
metrics = ["precision", "recall", "density", "coverage"]
classes = tuple(range(4))

# call the results.
path_list = glob.glob("../data/results/*/*GAN_evaluation*")
path_list.sort()
model_name = []
for path in path_list:
    model_name.append(os.path.basename(path)[:3]+os.path.basename(path)[18:-4])
model_name

['01_conventionalKL_unrolledk1_idtreg0',
 '01_proposedKL_unrolledk1_idtreg0',
 '01_proposedKL_unrolledk5_idtreg0.5',
 '02_singleD',
 '03_proposedKL',
 '05_pretrainedE']

# Quantitative Result
---
Let's observe the results from different view points.

---

---
- evaluation scores among selected classes and a selected model with different feature extractors

---

---
- **row**: evaluation metrics
- **column**: feature extractors

---

In [14]:
###############################################
############ adjustable parameters ############

source_label = 3 # source domain code
target_label = 2 # target domain code
model_idx = 0 # the index of the model name list

###############################################
###############################################

print(model_name[model_idx])
print("")
GAN_eval = pickle_load(path_list[model_idx])
scores = []
for fe in fe_list:
    for metric in metrics:
        scores.append(np.mean(GAN_eval[fe][source_label][target_label][metric]))
pd.DataFrame(np.array(scores).reshape(len(fe_list),4), index=fe_list, columns=metrics)

01_conventionalKL_unrolledk1_idtreg0



,precision,recall,density,coverage
vgg-initialization,0.67591,0.0000,0.530830,0.00485
vgg-ImageNet,0.65852,0.0000,0.578194,0.01662
vgg-CelebA,0.98711,0.0318,1.046208,0.06915


---
- **row**: classes
- **column**: classes

---

In [15]:
###############################################
############ adjustable parameters ############

fe = fe_list[0] # feature extractor
metric = "coverage" # evaluation metric
model_idx = 0 # the index of the model name list

###############################################
###############################################

print(fe)
print(model_name[model_idx])
print("")
GAN_eval = pickle_load(path_list[model_idx])
scores = []
for source_label in classes:
    for target_label in classes:
        scores.append(np.mean(GAN_eval[fe][source_label][target_label][metric]))
pd.DataFrame(np.array(scores).reshape(4,4), index=classes, columns=classes)

vgg-initialization
01_conventionalKL_unrolledk1_idtreg0



,0,1,2,3
0,0.00623,0.00763,0.00754,0.00919
1,0.00519,0.00670,0.00651,0.00611
2,0.01026,0.01191,0.00786,0.00958
3,0.00774,0.00911,0.00485,0.00686


---
- **row**: evaluation metrics
- **column**: model conditions

---

In [17]:
###############################################
############ adjustable parameters ############

source_label = 3 # source domain code
target_label = 2 # target domain code
fe = fe_list[2] # feature extractor

###############################################
###############################################

print(fe)
print("")
scores = []
for path in path_list:
    GAN_eval = pickle_load(path)
    for metric in metrics:
        scores.append(np.mean(GAN_eval[fe][source_label][target_label][metric]))
pd.DataFrame(np.array(scores).reshape(len(path_list),4), index=model_name, columns=metrics)

vgg-CelebA



,precision,recall,density,coverage
01_conventionalKL_unrolledk1_idtreg0,0.98711,0.03180,1.046208,0.06915
01_proposedKL_unrolledk1_idtreg0,0.98949,0.25546,0.957976,0.28472
01_proposedKL_unrolledk5_idtreg0.5,0.99173,0.25859,0.985018,0.25969
02_singleD,0.99312,0.13711,0.991170,0.17783
03_proposedKL,0.99384,0.14883,0.989042,0.17812
05_pretrainedE,0.99453,0.32016,1.020186,0.38889


---
- **row**: evaluation metrics
- **column**: model conditions

---

In [18]:
###############################################
############ adjustable parameters ############

fe = fe_list[0] # feature extractor

###############################################
###############################################

print(fe)
print("")
all_score = []
for source_label in classes:
    for target_label in classes:
        scores = []
        for path in path_list:
            GAN_eval = pickle_load(path)
            for metric in metrics:
                scores.append(np.mean(GAN_eval[fe][source_label][target_label][metric]))
        all_score.append(np.array(scores).reshape(len(path_list), 4))
pd.DataFrame(np.mean(np.array(all_score), axis=0), index=model_name, columns=metrics)

vgg-initialization



,precision,recall,density,coverage
01_conventionalKL_unrolledk1_idtreg0,0.830858,0.000000,0.770116,0.007704
01_proposedKL_unrolledk1_idtreg0,0.830172,0.000016,0.688579,0.061609
01_proposedKL_unrolledk5_idtreg0.5,0.887107,0.000128,0.959872,0.115728
02_singleD,0.867024,0.000106,0.891322,0.075316
03_proposedKL,0.851789,0.001673,0.818887,0.192123
05_pretrainedE,0.872410,0.000000,0.921793,0.012883


---
- **row**: classes; **s** means source domain code and **t** means target domain code.
- **column**: model conditions

---

In [20]:
###############################################
############ adjustable parameters ############

fe = fe_list[2] # feature extractor
metric = "coverage" # evaluation metric

###############################################
###############################################

print(fe)
print("")
columns = []
for i in classes:
    for j in classes:
        columns.append((f"s:{i}", f"t:{j}"))
columns = pd.MultiIndex.from_tuples(columns)

scores = []
for path in path_list:
    GAN_eval = pickle_load(path)
    for source_label in classes:
        for target_label in classes:
            scores.append(np.round(np.mean(GAN_eval[fe][source_label][target_label][metric]),4))
pd.DataFrame(np.array(scores).reshape(len(path_list),16), index=model_name, columns=columns)

vgg-CelebA



s:0                             s:1  \
                                         t:0     t:1     t:2     t:3     t:0   
01_conventionalKL_unrolledk1_idtreg0  0.0335  0.0552  0.0709  0.1004  0.0572   
01_proposedKL_unrolledk1_idtreg0      0.1686  0.2000  0.2305  0.2227  0.2073   
01_proposedKL_unrolledk5_idtreg0.5    0.2561  0.2592  0.2657  0.2844  0.2986   
02_singleD                            0.1675  0.1687  0.1273  0.2141  0.1850   
03_proposedKL                         0.1979  0.2142  0.1601  0.2132  0.1955   
05_pretrainedE                        0.2521  0.2798  0.2524  0.3892  0.3103   

                                                                 s:2          \
                                         t:1     t:2     t:3     t:0     t:1   
01_conventionalKL_unrolledk1_idtreg0  0.0275  0.0750  0.0533  0.0826  0.1031   
01_proposedKL_unrolledk1_idtreg0      0.1752  0.2547  0.1704  0.2030  0.2704   
01_proposedKL_unrolledk5_idtreg0.5    0.2476  0.2522  0.2411  0.2245  0.3012   
02_singleD                            0.1685  0.1167  0.1590  0.1170  0.1102   
03_proposedKL                         0.2132  0.1411  0.1894  0.1002  0.1658   
05_pretrainedE                        0.2054  0.2759  0.2316  0.2582  0.3536   

                                                         s:3                  \
                                         t:2     t:3     t:0     t:1     t:2   
01_conventionalKL_unrolledk1_idtreg0  0.0338  0.0822  0.0905  0.0657  0.0691   
01_proposedKL_unrolledk1_idtreg0      0.2271  0.2455  0.2915  0.1872  0.2847   
01_proposedKL_unrolledk5_idtreg0.5    0.2365  0.2846  0.2509  0.2302  0.2597   
02_singleD                            0.1687  0.1709  0.1230  0.1115  0.1778   
03_proposedKL                         0.1926  0.2015  0.1218  0.1276  0.1781   
05_pretrainedE                        0.3441  0.4398  0.3141  0.2430  0.3889   

                                              
                                         t:3  
01_conventionalKL_unrolledk1_idtreg0  0.0268  
01_proposedKL_unrolledk1_idtreg0      0.1809  
01_proposedKL_unrolledk5_idtreg0.5    0.2272  
02_singleD                            0.1570  
03_proposedKL                         0.2058  
05_pretrainedE                        0.2541